In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error





# Charger les données
df = pd.read_excel("Total_Presents_Final.xlsx")
print("Données initiales :")
print(df.info())

# Compléter les années manquantes
df['Annee'] = df['Annee'].fillna(method='ffill')

# Supprimer les lignes avec des semaines ou des présences manquantes
df = df.dropna(subset=['Semaines', 'Presences'])

# Convertir la colonne "Semaines" pour extraire uniquement le numéro de semaine
df['Semaines'] = df['Semaines'].str.extract(r'(\d+)').astype(int)

# Fonction pour convertir "Année" et "Semaines" en une date (lundi de la semaine donnée)
def year_week_to_date(year, week):
    """Convertit une année et un numéro de semaine en une date (lundi de cette semaine)."""
    first_day = datetime(year, 1, 1)
    first_week_start = first_day + timedelta(days=(7 - first_day.weekday()))  # Premier lundi
    return first_week_start + timedelta(weeks=week - 1)

# Appliquer la fonction pour créer une colonne "Date"
df['Date'] = df.apply(lambda row: year_week_to_date(int(row['Annee']), int(row['Semaines'])), axis=1)

# Remplacer l'index par la colonne "Date"
df.set_index('Date', inplace=True)

# Garder uniquement la colonne des présences pour le modèle
time_series = df['Presences']

# Diviser les données en train (historique) et test (futur)
train = time_series[:-10]
test = time_series[-10:]

# Construire et ajuster le modèle
p, d, q = 2, 1, 2
P, D, Q, s =1, 1, 1, 52
model = SARIMAX(train, order=(p, d, q), seasonal_order=(P, D, Q, s), enforce_stationarity=False, enforce_invertibility=False)
sarima_model = model.fit(disp=False)

# Générer des prédictions pour l'ensemble test
forecast = sarima_model.get_forecast(steps=len(test))
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()  # AJOUT : intervalle de confiance pour le test

# Calcul du MAE (Mean Absolute Error)
mae = mean_absolute_error(test, forecast_mean)
mae_percentage = (mae / test.mean()) * 100  # Convertir en pourcentage

# Affichage des métriques
print(f"Erreur absolue moyenne (MAE) : {mae:.2f}")
print(f"Précision moyenne : {100 - mae_percentage:.2f}%")  # La précision est 100% - MAE%

# Prédictions futures (30 prochaines semaines)
future_forecast = sarima_model.get_forecast(steps=30)
future_index = pd.date_range(start=time_series.index[-1] + timedelta(weeks=1), periods=30, freq='W-MON')
future_mean = future_forecast.predicted_mean
future_ci = future_forecast.conf_int()  # AJOUT : intervalle de confiance pour les prédictions futures

# Affichage des prédictions et des intervalles de confiance
plt.figure(figsize=(12, 6))
plt.plot(train, label='Données réelles (Entraînement)', color='blue')
plt.plot(test, label='Données réelles (Test)', color='orange')
#plt.plot(forecast_mean.index, forecast_mean, label='Prédictions (Test)', color='green')
plt.fill_between(forecast_mean.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='green', alpha=0.2)
plt.plot(future_index, future_mean, label='Prédictions futures', color='red')
plt.fill_between(future_index, future_ci.iloc[:, 0], future_ci.iloc[:, 1], color='red', alpha=0.2)
plt.xlim(train.index.min(), future_index.max())
plt.title("Prédictions des présences avec SARIMA")
plt.xlabel("Date")
plt.ylabel("Présences")
plt.legend()
plt.grid()
plt.show()

# Affichage des prédictions futures
print("Prédictions pour les 30 prochaines semaines :")
for date, pred in zip(future_index, future_mean):
    print(f"{date.strftime('%Y-%m-%d')} : {pred:.2f} présences")

Données initiales :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Annee      3 non-null      float64
 1   Semaines   143 non-null    object 
 2   Presences  143 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.5+ KB
None


/tmp/ipykernel_153/3082998969.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Annee'] = df['Annee'].fillna(method='ffill')
/opt/conda/envs/anaconda-2024.02-py310/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/envs/anaconda-2024.02-py310/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/envs/anaconda-2024.02-py310/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for v